In [ ]:
from pymongo import MongoClient
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from datetime import timedelta 

# 크롤링 참고(https://operstu1.tistory.com/87)

In [ ]:
# 지역별 관측소 이름 데이터
sido_data = {
  "서울특별시": "종로구",
  "경기도": "가평",
  "인천광역시": "신흥",  
  "강원도": "문막읍",
  "충청남도": "배방읍",
  "대전광역시": "대성동",
  "충청북도": "가덕면",
  "부산광역시": "광복동",  
  "울산광역시": "성남동",
  "대구광역시": "남산1동",
  "경상북도": "대도동",
  "경상남도": "경화동",
  "전라남도": "여천동(여수)",
  "광주광역시": "서석동",
  "전라북도": "노송동",
  "제주특별자치도": "연동",
  "세종특별자치시": "신흥동"
    }

# 지역 이름 추출
sido_keys= list(sido_data.keys())

In [ ]:
def parse() :
    try:
        # 24:00 오류 방지
        date = item.find('datatime').get_text()
        if '24:00' in date :
            date = date.replace('24:00', '00:00')
            date = datetime.strptime(date, '%Y-%m-%d %H:%M')
            datatime = date + timedelta(days=1)
        else:
            datatime = datetime.strptime(date, '%Y-%m-%d %H:%M')
        region: s

        o3value = item.find('o3value').get_text()
        no2value = item.find('no2value').get_text()
        covalue = item.find('covalue').get_text()
        pm10value = item.find('pm10value').get_text()

        return {
            'datatime':datatime,
            'region': s,

            'o3value':o3value,
            'no2value':no2value,
            'covalue':covalue,
            'pm10value':pm10value
        }
    except AttributeError as e:
        return {
            'datatime':None,
            'region': s,

            'o3value':None,
            'no2value':None,
            'covalue':None,
            'pm10value':None
        }

In [ ]:
# MongoDB 연결용
db = mongo.dustDrive_prod

# 실시간 측정 정보_DAILY

In [ ]:
# https://www.data.go.kr/data/15073861/openapi.do
# 측정소별 실시간 측정 정보

url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty?'


returnType = '&returnType=xml'
numOfRows = '&numOfRows=100'
pageNo = '&pageNo=1'
dataTerm = '&dataTerm=DAILY'

In [ ]:
for s in sido_keys:
    row = []
    stationName = '&stationName='+ sido_data[s]

    # print(url+api_key+returnType+numOfRows+pageNo+stationName+dataTerm)
    result = requests.get(url+api_key+returnType+numOfRows+pageNo+stationName+dataTerm)
    
    bs = BeautifulSoup(result.text, 'lxml')
    items = bs.find_all('item')

    for item in items:
        row.append(parse())

    # DB 업로드
    result = db.dust_info.insert_many(row)

# 실시간 측정 정보_3MONTH

In [ ]:
url = 'http://apis.data.go.kr/B552584/ArpltnInforInqireSvc/getMsrstnAcctoRltmMesureDnsty?'

returnType = '&returnType=xml'
numOfRows = '&numOfRows=3000'
pageNo = '&pageNo=1'
dataTerm = '&dataTerm=3MONTH'

In [ ]:
for s in sido_keys:
    row = []
    stationName = '&stationName='+ sido_data[s]

    result = requests.get(url+api_key+returnType+numOfRows+pageNo+stationName+dataTerm)
    

    bs = BeautifulSoup(result.text, 'lxml')
    items = bs.find_all('item')

    for item in items:
        row.append(parse())

    db.dust_info_api.insert_many(row)
    print(s)
    

In [ ]:
row

# 대기오염 데이터_2022

In [ ]:
# 2022년 충청남도 배방읍 관측소 대기오염 데이터 파일

for i in range(12):
    globals()['df{}'.format(i)] = pd.read_csv(f'./csv_data/data_past_day ({i}).csv', encoding='cp949')

In [ ]:
# 필요없는 행 삭제

df0 = df0.drop(index=[0, len(df0)-1], axis = 0)
df1 = df1.drop(index=[0, len(df1)-1], axis = 0)
df2 = df2.drop(index=[0, len(df2)-1], axis = 0)
df3 = df3.drop(index=[0, len(df3)-1], axis = 0)
df4 = df4.drop(index=[0, len(df4)-1], axis = 0)
df5 = df5.drop(index=[0, len(df5)-1], axis = 0)
df6 = df6.drop(index=[0, len(df6)-1], axis = 0)
df7 = df7.drop(index=[0, len(df7)-1], axis = 0)
df8 = df8.drop(index=[0, len(df8)-1], axis = 0)
df9 = df9.drop(index=[0, len(df9)-1], axis = 0)
df10 = df10.drop(index=[0, len(df10)-1], axis = 0)
df11 = df11.drop(index=[0, len(df11)-1], axis = 0)

In [ ]:
df_list = [df0,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11]

df2022 = pd.concat(df_list, ignore_index=True)

In [ ]:
# 데이터 컬럼명 변경
df2022.columns = ['datatime', 'pm10value', 'pm25value', 'o3value', 'no2value', 'covalue', 'so2value']

In [ ]:
# 지역 컬럼 추가
region = pd.DataFrame(['충청남도']*len(df2022))
df2022.insert(1, 'region', region)

# 날짜 형식 datetime으로 변경
df2022['datatime'] = pd.to_datetime(df2022['datatime'])

# 필요없는 컬럼 삭제
df2022.drop(['pm25value', 'so2value'], axis=1, inplace=True)

# 딕셔너리 형 변환
df2022_dict = df2022.to_dict('records')

In [ ]:
# DB 업로드
db.dust_info_csv.insert_many(df2022_dict)